In [2]:
from mdcrow import MDCrow

In [3]:
prompt3 = "Download the PDB file for protein 1GZX. Then, analyze the secondary structure of the protein and provide information on how many helices, sheets, and other components are present. Get the gene names for this protein."
llm_var = "claude-3-5-sonnet-20240620"
tools = "all"
agent = MDCrow(agent_type="Structured", model=llm_var, top_k_tools=tools, use_memory=False)
print (llm_var)

claude-3-5-sonnet-20240620


In [4]:
agent.run(prompt3)

Thought: To answer this question, I need to follow these steps:
1. Download the PDB file for protein 1GZX
2. Analyze the secondary structure of the protein
3. Get the gene names for this protein

Let's start by downloading the PDB file.

Action:
```
{
    "action": "PDBFileDownloader",
    "action_input": "1GZX"
}
```PDB file found with this ID: 1GZX
Thought: Great, we have successfully downloaded the PDB file for protein 1GZX. Now, let's analyze the secondary structure of the protein using the ComputeDSSP tool. This tool will provide information about the secondary structure elements, including helices and sheets.

Action:
```
{
    "action": "ComputeDSSP",
    "action_input": {
        "traj_file": "1GZX_201328",
        "top_file": "1GZX_201328",
        "target_frames": "all"
    }
}
```

Thought: We've successfully analyzed the secondary structure of the protein 1GZX. Now, let's get the gene names for this protein using the GetGeneNames tool. After that, we'll have all the informa

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [5]:
registry = agent.path_registry
all_paths = registry.list_path_names_and_descriptions()
print (all_paths)
assert "1GZX" in all_paths
file_id = all_paths.split("Files found in registry: ")[1].split(":")[0]

Files found in registry: 1GZX_201328: PDB file downloaded from RSCB, PDBFile ID: 1GZX_201328, rec0_201332: dssp values for trajectory with id: 1GZX_201328


In [6]:
import mdtraj as md
file_path = registry.get_mapped_path(file_id)
traj = md.load(file_path)
top = traj.topology

secondary_structure = md.compute_dssp(traj,simplified=True)
print("Number of residues in sheets: ",len([i for i in secondary_structure[0] if i == 'E']))
print("Number of residues in helices: ",len([i for i in secondary_structure[0] if i == 'H']))
print("Number of residues in coils: ",len([i for i in secondary_structure[0] if i == 'C']))

Number of residues in sheets:  0
Number of residues in helices:  444
Number of residues in coils:  130


In [10]:
from mdcrow.tools.base_tools import GetGeneNames
GetGeneNames().run("1GZX")